In [1]:
# 加载本地env的key
import dotenv
%load_ext dotenv
%dotenv

# 添加异步处理
import nest_asyncio
nest_asyncio.apply()

In [2]:
from llama_index.core import SimpleDirectoryReader

documents = SimpleDirectoryReader(input_files=["./datasets/maiyouweng.pdf"]).load_data()

from llama_index.core.node_parser import SentenceSplitter

splitter = SentenceSplitter(chunk_size=1024)

nodes = splitter.get_nodes_from_documents(documents)

In [3]:
from llama_index.core import Settings
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding

# LLM model
Settings.llm = OpenAI(model="gpt-3.5-turbo")
# embedding model
Settings.embed_model = OpenAIEmbedding(model="text-embedding-ada-002")

In [4]:
from llama_index.core import SummaryIndex, VectorStoreIndex

# summary index
summary_index = SummaryIndex(nodes)
# vector store index
vector_index = VectorStoreIndex(nodes)

# summary query engine
summary_query_engine = summary_index.as_query_engine(
    response_mode="tree_summarize",
    use_async=True,
)

# vector query engine
vector_query_engine = vector_index.as_query_engine()

In [5]:
llm = OpenAI(model="gpt-3.5-turbo", temperature=0)

In [6]:
from llama_index.core.tools import QueryEngineTool

In [7]:
summary_tool = QueryEngineTool.from_defaults(
    query_engine=summary_query_engine,
    description=(
        "用于生成关于卖油翁课文摘要"
    ),
)

vector_tool = QueryEngineTool.from_defaults(
    query_engine=vector_query_engine,
    description=(
        "用户常规检索卖油翁索引"
    ),
)

In [8]:
from llama_index.core.agent import FunctionCallingAgentWorker
from llama_index.core.agent import AgentRunner

agent_worker = FunctionCallingAgentWorker.from_tools(
    tools=[vector_tool, summary_tool], 
    llm=llm, 
    verbose=True
)
agent = AgentRunner(agent_worker)

In [9]:
response = agent.query(
    "卖油翁的故事是从什么典故发生的?"
) 

Added user message to memory: 卖油翁的故事是从什么典故发生的?
=== Calling Function ===
Calling function: query_engine_tool with args: {"input": "\u5356\u6cb9\u7fc1\u7684\u6545\u4e8b\u662f\u4ece\u4ec0\u4e48\u5178\u6545\u53d1\u751f\u7684?"}
=== Function Output ===
陈尧咨和卖油翁的故事是从欧阳修的《卖油翁》中发生的。
=== LLM Response ===
卖油翁的故事是从欧阳修的《卖油翁》中发生的。


In [11]:
response = agent.chat(
    "解释一下卖油翁有什么启发，为什么要学习？"
)

print(str(response))

Added user message to memory: 解释一下卖油翁有什么启发，为什么要学习？
=== Calling Function ===
Calling function: query_engine_tool with args: {"input": "\u5356\u6cb9\u7fc1\u6709\u4ec0\u4e48\u542f\u53d1\uff0c\u4e3a\u4ec0\u4e48\u8981\u5b66\u4e60\uff1f"}
=== Function Output ===
The story of "卖油翁" serves as an inspiration by highlighting the importance of diligence and practice in honing one's skills. It emphasizes the concept of "熟能生巧," which means that through continuous practice and effort, one can become proficient in a particular skill. The character of 卖油翁 demonstrates the value of humility, perseverance, and the continuous pursuit of excellence. This story encourages learning by showcasing the rewards of dedicated practice and the wisdom gained from experience.
=== LLM Response ===
卖油翁的故事启示我们关于勤奋和实践的重要性。它强调了“熟能生巧”的概念，即通过持续的练习和努力，人们可以在特定的技能上变得熟练。卖油翁这个角色展示了谦卑、毅力和不断追求卓越的价值。这个故事鼓励学习，展示了专心练习所带来的回报以及从经验中获得的智慧。
卖油翁的故事启示我们关于勤奋和实践的重要性。它强调了“熟能生巧”的概念，即通过持续的练习和努力，人们可以在特定的技能上变得熟练。卖油翁这个角色展示了谦卑、毅力和不断追求卓越的价值。这个故事鼓励学习

In [12]:
response = agent.chat(
    "我最后一个问题是什么？"
)

print(str(response))

Added user message to memory: 我最后一个问题是什么？
=== LLM Response ===
您的最后一个问题是关于卖油翁的启发和学习的相关问题。
您的最后一个问题是关于卖油翁的启发和学习的相关问题。


In [13]:
agent_worker = FunctionCallingAgentWorker.from_tools(
    [vector_tool, summary_tool], 
    llm=llm, 
    verbose=True
)
agent = AgentRunner(agent_worker)

In [14]:
task = agent.create_task(
    "解释一下卖油翁有什么启发？"
    "为什么现代还要学习？"
)

In [15]:
completed_steps = agent.get_completed_steps(task.task_id)

print(f"Number of completed steps for tasksID {task.task_id} is {len(completed_steps)}")

if len(completed_steps) > 0:
    print(completed_steps[0].output.sources[0].raw_output)

Number of completed steps for tasksID d5a52616-d366-4f51-bd7c-c0ec6d948c84 is 0


In [16]:
upcoming_steps = agent.get_upcoming_steps(task.task_id)
print(f"Number of completed steps for tasksID {task.task_id} is {len(upcoming_steps)}")

if len(upcoming_steps) > 0:
    print(upcoming_steps[0].input)

Number of completed steps for tasksID d5a52616-d366-4f51-bd7c-c0ec6d948c84 is 1
解释一下卖油翁有什么启发？为什么现代还要学习？


In [21]:
step_output = agent.run_step(task.task_id)

=== LLM Response ===
卖油翁倾油的技巧强调了“技能源自实践”的原则和对特定技能专业知识的重要性。它强调熟练不仅仅是天赋的结果，而更多是持续的实践和经验积累的结果。


In [22]:
print(step_output.is_last)

True


In [19]:
task = agent.create_task(
    "解释一下卖油翁有什么启发？"
    "为什么现代还要学习？"
)

In [20]:
step_output = agent.run_step(
    task.task_id, input="卖油翁倾油的技巧强调了什么？"
)

Added user message to memory: 卖油翁倾油的技巧强调了什么？
=== Calling Function ===
Calling function: query_engine_tool with args: {"input": "\u5356\u6cb9\u7fc1\u503e\u6cb9\u7684\u6280\u5de7\u5f3a\u8c03\u4e86\u4ec0\u4e48\uff1f"}
=== Function Output ===
The technique of pouring oil by the oil seller emphasizes the principle of "skill comes from practice" and the importance of expertise in a particular skill. It highlights the idea that proficiency is not just a result of innate talent, but rather the outcome of continuous practice and experience.


In [23]:
response = agent.finalize_response(task.task_id)
print(str(response))

卖油翁倾油的技巧强调了“技能源自实践”的原则和对特定技能专业知识的重要性。它强调熟练不仅仅是天赋的结果，而更多是持续的实践和经验积累的结果。
